In [1]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 3.9 MB/s eta 0:00:00


In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings('ignore')

In [42]:
data = pd.read_csv('transactions.csv')

In [43]:
data.shape

(199999, 10)

In [44]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,8,CASH_OUT,158007.12,C424875646,0.00,0.00,C1298177219,474016.32,1618631.97,0
1,236,CASH_OUT,457948.30,C1342616552,0.00,0.00,C1323169990,2720411.37,3178359.67,0
2,37,CASH_IN,153602.99,C900876541,11160428.67,11314031.67,C608741097,3274930.56,3121327.56,0
3,331,CASH_OUT,49555.14,C177696810,10865.00,0.00,C462716348,0.00,49555.14,0
4,250,CASH_OUT,29648.02,C788941490,0.00,0.00,C1971700992,56933.09,86581.10,0


In [45]:
data = data.drop(['nameOrig', 'nameDest'], axis=1)

In [46]:
data.head()

,step,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud
0,8,CASH_OUT,158007.12,0.00,0.00,474016.32,1618631.97,0
1,236,CASH_OUT,457948.30,0.00,0.00,2720411.37,3178359.67,0
2,37,CASH_IN,153602.99,11160428.67,11314031.67,3274930.56,3121327.56,0
3,331,CASH_OUT,49555.14,10865.00,0.00,0.00,49555.14,0
4,250,CASH_OUT,29648.02,0.00,0.00,56933.09,86581.10,0


In [47]:
data = data.dropna()

In [48]:
X = data.drop('isFraud', axis=1)
y = data['isFraud']

In [49]:
categorical_features = ['type']
numerical_features = ['amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


In [52]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(preprocessor.fit_transform(X_train), y_train)

In [53]:
X_test_transformed = preprocessor.transform(X_test)

In [54]:
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=50),
    'Gradient Boosting (XGBoost)': XGBClassifier(random_state=42, max_depth=6, eval_metric='logloss'),
    'Naive Bayes': GaussianNB()
}

In [67]:
results = {}
for name, model in models.items():
    # Train the model
    model.fit(X_train_resampled, y_train_resampled)

    # Predict on test set
    y_pred = model.predict(X_test_transformed)
    y_proba = model.predict_proba(X_test_transformed)[:, 1] if hasattr(model, 'predict_proba') else model.decision_function(X_test_transformed)

    # Calculate metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    auc_score = roc_auc_score(y_test, y_proba)
    cm = confusion_matrix(y_test, y_pred)
    acc = accuracy_score(y_test, y_pred)

    results[name] = {
        'Accuracy': acc,
        'Precision (Fraud)': report['1']['precision'],
        'Recall (Fraud)': report['1']['recall'],
        'F1-Score (Fraud)': report['1']['f1-score'],
        'AUC-ROC': auc_score,
        'Confusion Matrix': cm
    }

In [68]:
from sklearn.metrics import classification_report

print("\n Classification Report for Each Model:\n")

for name, model in models.items():
    print(f"=== {name} ===")

    # Predict on test set
    y_pred = model.predict(X_test_transformed)

    # Print full classification report
    print(classification_report(y_test, y_pred, digits=4))
    print("-" * 60)



 Classification Report for Each Model:

=== Logistic Regression ===
              precision    recall  f1-score   support

           0     1.0000    0.9365    0.9672     59915
           1     0.0216    0.9882    0.0423        85

    accuracy                         0.9366     60000
   macro avg     0.5108    0.9624    0.5048     60000
weighted avg     0.9986    0.9366    0.9659     60000

------------------------------------------------------------
=== Decision Tree ===
              precision    recall  f1-score   support

           0     0.9999    0.9985    0.9992     59915
           1     0.4639    0.9059    0.6135        85

    accuracy                         0.9984     60000
   macro avg     0.7319    0.9522    0.8064     60000
weighted avg     0.9991    0.9984    0.9986     60000

------------------------------------------------------------
=== Random Forest ===
              precision    recall  f1-score   support

           0     0.9998    0.9984    0.9991     59915
  

In [70]:
for name, metrics in results.items():
    print(f"\nModel: {name}")
    print(f"Accuracy: {metrics['Accuracy']:.4f}")
    print(f"Precision (Fraud): {metrics['Precision (Fraud)']:.4f}")
    print(f"Recall (Fraud): {metrics['Recall (Fraud)']:.4f}")
    print(f"F1-Score (Fraud): {metrics['F1-Score (Fraud)']:.4f}")
    print(f"AUC-ROC: {metrics['AUC-ROC']:.4f}")
    print(f"Confusion Matrix:\n{metrics['Confusion Matrix']}")


Model: Logistic Regression
Accuracy: 0.9366
Precision (Fraud): 0.0216
Recall (Fraud): 0.9882
F1-Score (Fraud): 0.0423
AUC-ROC: 0.9961
Confusion Matrix:
[[56112  3803]
 [    1    84]]

Model: Decision Tree
Accuracy: 0.9984
Precision (Fraud): 0.4639
Recall (Fraud): 0.9059
F1-Score (Fraud): 0.6135
AUC-ROC: 0.9522
Confusion Matrix:
[[59826    89]
 [    8    77]]

Model: Random Forest
Accuracy: 0.9983
Precision (Fraud): 0.4471
Recall (Fraud): 0.8941
F1-Score (Fraud): 0.5961
AUC-ROC: 0.9935
Confusion Matrix:
[[59821    94]
 [    9    76]]

Model: Gradient Boosting (XGBoost)
Accuracy: 0.9978
Precision (Fraud): 0.3821
Recall (Fraud): 0.9529
F1-Score (Fraud): 0.5455
AUC-ROC: 0.9992
Confusion Matrix:
[[59784   131]
 [    4    81]]

Model: Naive Bayes
Accuracy: 0.3979
Precision (Fraud): 0.0022
Recall (Fraud): 0.9529
F1-Score (Fraud): 0.0045
AUC-ROC: 0.9026
Confusion Matrix:
[[23792 36123]
 [    4    81]]


In [75]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# ------------------ PART 1: CLASSIFICATION REPORT ANALYSIS ------------------

model_scores = {}

print("\n📋 Classification Report Evaluation:\n")

for name, model in models.items():
    y_pred = model.predict(X_test_transformed)
    report = classification_report(y_test, y_pred, output_dict=True)

    fraud_metrics = report['1']  # class '1' = fraud
    f1 = fraud_metrics['f1-score']
    recall = fraud_metrics['recall']
    precision = fraud_metrics['precision']

    model_scores[name] = {
        'F1-Score (Fraud)': f1,
        'Recall (Fraud)': recall,
        'Precision (Fraud)': precision
    }

    print(f"=== {name} ===")
    print(f"Precision (Fraud): {precision:.4f}")
    print(f"Recall (Fraud):    {recall:.4f}")
    print(f"F1-Score (Fraud):  {f1:.4f}")
    print("-" * 40)

# Select best model based on F1-Score
best_model_f1 = max(model_scores.items(), key=lambda x: x[1]['F1-Score (Fraud)'])
print(f"\n✅ Best Model Based on F1-Score (Fraud): {best_model_f1[0]}")
print(f"F1-Score (Fraud): {best_model_f1[1]['F1-Score (Fraud)']:.4f}")

# ------------------ PART 2: OVERALL BEST MODEL USING MULTIPLE METRICS ------------------

# Important metrics to evaluate from results
important_metrics = [
    'Accuracy',
    'Precision (Fraud)',
    'Recall (Fraud)',
    'F1-Score (Fraud)',
    'AUC-ROC'
]

# Create a DataFrame from results dictionary
results_df = pd.DataFrame(results).T
metrics_df = results_df[important_metrics]

# Normalize all metrics using MinMaxScaler (to range [0,1])
scaler = MinMaxScaler()
normalized_metrics = pd.DataFrame(
    scaler.fit_transform(metrics_df),
    columns=important_metrics,
    index=metrics_df.index
)

# Compute average score for each model
normalized_metrics['Overall Score'] = normalized_metrics.mean(axis=1)

# Get best model based on overall score
best_overall_model = normalized_metrics['Overall Score'].idxmax()
best_overall_score = normalized_metrics.loc[best_overall_model, 'Overall Score']

# ------------------ PRINT FINAL RESULTS ------------------

print("\n📊 Combined Metric Ranking (Normalized Scores):")
print(normalized_metrics.sort_values('Overall Score', ascending=False))

print(f"\n🏆 Best Overall Model Based on All Metrics: {best_overall_model}")
print(f"Overall Combined Score: {best_overall_score:.4f}")



📋 Classification Report Evaluation:

=== Logistic Regression ===
Precision (Fraud): 0.0216
Recall (Fraud):    0.9882
F1-Score (Fraud):  0.0423
----------------------------------------
=== Decision Tree ===
Precision (Fraud): 0.4639
Recall (Fraud):    0.9059
F1-Score (Fraud):  0.6135
----------------------------------------
=== Random Forest ===
Precision (Fraud): 0.4471
Recall (Fraud):    0.8941
F1-Score (Fraud):  0.5961
----------------------------------------
=== Gradient Boosting (XGBoost) ===
Precision (Fraud): 0.3821
Recall (Fraud):    0.9529
F1-Score (Fraud):  0.5455
----------------------------------------
=== Naive Bayes ===
Precision (Fraud): 0.0022
Recall (Fraud):    0.9529
F1-Score (Fraud):  0.0045
----------------------------------------

✅ Best Model Based on F1-Score (Fraud): Decision Tree
F1-Score (Fraud): 0.6135

📊 Combined Metric Ranking (Normalized Scores):
                             Accuracy  Precision (Fraud)  Recall (Fraud)  \
Gradient Boosting (XGBoost)  0.9989

In [76]:
from sklearn.metrics import classification_report

model_scores = {}

print("\n📋 Classification Report Evaluation:\n")

for name, model in models.items():
    y_pred = model.predict(X_test_transformed)
    report = classification_report(y_test, y_pred, output_dict=True)

    fraud_metrics = report['1']  # class '1' = fraud
    f1 = fraud_metrics['f1-score']
    recall = fraud_metrics['recall']
    precision = fraud_metrics['precision']

    model_scores[name] = {
        'F1-Score (Fraud)': f1,
        'Recall (Fraud)': recall,
        'Precision (Fraud)': precision
    }

    print(f"=== {name} ===")
    print(f"Precision (Fraud): {precision:.4f}")
    print(f"Recall (Fraud):    {recall:.4f}")
    print(f"F1-Score (Fraud):  {f1:.4f}")
    print("-" * 40)

# Select best model based on F1-Score (Fraud)
best_model = max(model_scores.items(), key=lambda x: x[1]['F1-Score (Fraud)'])

print(f"\n✅ Best Model Based on F1-Score (Fraud): {best_model[0]}")
print(f"F1-Score (Fraud): {best_model[1]['F1-Score (Fraud)']:.4f}")



📋 Classification Report Evaluation:

=== Logistic Regression ===
Precision (Fraud): 0.0216
Recall (Fraud):    0.9882
F1-Score (Fraud):  0.0423
----------------------------------------
=== Decision Tree ===
Precision (Fraud): 0.4639
Recall (Fraud):    0.9059
F1-Score (Fraud):  0.6135
----------------------------------------
=== Random Forest ===
Precision (Fraud): 0.4471
Recall (Fraud):    0.8941
F1-Score (Fraud):  0.5961
----------------------------------------
=== Gradient Boosting (XGBoost) ===
Precision (Fraud): 0.3821
Recall (Fraud):    0.9529
F1-Score (Fraud):  0.5455
----------------------------------------
=== Naive Bayes ===
Precision (Fraud): 0.0022
Recall (Fraud):    0.9529
F1-Score (Fraud):  0.0045
----------------------------------------

✅ Best Model Based on F1-Score (Fraud): Decision Tree
F1-Score (Fraud): 0.6135


In [77]:
best_model = max(results.items(), key=lambda x: x[1]['F1-Score (Fraud)'])
print(f"\nBest Model: {best_model[0]}")
print(f"Best F1-Score (Fraud): {best_model[1]['F1-Score (Fraud)']:.4f}")


Best Model: Decision Tree
Best F1-Score (Fraud): 0.6135


In [78]:
best_model = max(results.items(), key=lambda x: x[1]['Accuracy'])
print(f"\nBest Model: {best_model[0]}")
print(f"Best Accuracy: {best_model[1]['Accuracy']:.4f}")


Best Model: Decision Tree
Best Accuracy: 0.9984
